In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb

import data_utils as data


tt = gp.Model('IIITB Exams Timetable')

31
aasmaan.gupta@iiitb.ac.in
samarjeet.wankhade@iiitb.ac.in
Tanmay.Jain015@iiitb.ac.in
Shreyan.Gupta@iiitb.ac.in
Shubham.Agarwal@iiitb.ac.in
Nimish.Khandeparkar@iiitb.ac.in
Vidhu.Arora@iiitb.ac.in
Mayank.Sharma@iiitb.ac.in
Ayush.Singh@iiitb.ac.in
Rithik.Bansal@iiitb.ac.in
Yash.Bansal513@iiitb.ac.in
Kushal.Jenamani@iiitb.ac.in
Madhav.Patil@iiitb.ac.in
Ashashree.Sarma@iiitb.ac.in
Patelrishita.Dipak@iiitb.ac.in
Pranita.Ganguly@iiitb.ac.in
Shindedev.Hemravi@iiitb.ac.in
Suhan.Roy@iiitb.ac.in
AadilMohammad.Husain@iiitb.ac.in
fitriana.dewi@iiitb.ac.in
Ritish.Shrirao@iiitb.ac.in
Trupti.Khodwe@iiitb.ac.in
Harsh.Dhruv@iiitb.ac.in
Madhav.Girdhar@iiitb.ac.in
Divyam.Sareen@iiitb.ac.in
Prateek.Rath@iiitb.ac.in
Swetha.Murali@iiitb.ac.in
Daksh.Rajesh@iiitb.ac.in
Rutul.Patel@iiitb.ac.in
Nathan.Verghese@iiitb.ac.in
Ananthula.Reddy@iiitb.ac.in
Varnit.Mittal@iiitb.ac.in
Siddeshwar.Kagatikar@iiitb.ac.in
Manda.Kausthubh@iiitb.ac.in
Nikita.Kiran@iiitb.ac.in
Siddharth.Maramreddy@iiitb.ac.in
Srivatsa.Tarun@iii

## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
def exam_length(course:int):
    return 120

def exam_slots(course: int):
    # return (data.get_exam_duration(course)//15)

    return int(exam_length(course)//15)
#Dhruv's part:

# session_spans = {s: begin_end_pairs(session_length(s)) for s in range(n_sessions)}
# print(data.begin_end_pairs(exam_slots(1)))
exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [3]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if not data.overlap(b,e,t)]

In [4]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course

In [5]:


# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [6]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for c in range(data.n_courses)
#                           for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
#               for d in range(data.n_days)
#               for r in range(data.n_rooms)
#               for t in range(data.n_times)),
#               name = "room_capacity_constraint"
#              )

P = tt.addVars(
    [
        (c, r, b, e, d)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.INTEGER,
    name = 'P',
    lb = 0, 
    ub = 230,
)


for c,r,b,e,d in P:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(P[c,r,b,e,d] >=0, name = 'P1_constraint')
    tt.addConstr(P[c,r,b,e,d] <= room_capacity*E[c,b,e,d], name = 'P2_constraint')
    tt.addConstr(P[c,r,b,e,d] <= C[c,r]*(1-E[c,b,e,d]), name = 'P3_constraint')
    tt.addConstr(P[c,r,b,e,d] >= (C[c,r] - room_capacity)*(1-E[c,b,e,d]), name = 'P4_constraint')



tt.addConstrs(
    (gp.quicksum(P[c, r, b, e, d] 
                 for r in range(data.n_rooms) 
                 for b, e in covering_sessions(c, t)) <= data.get_room_capacity(r)
     for d in range(data.n_days) 
     for c in range(data.n_courses) 
     for t in range(data.n_times)),
    name='room_capacity_constraint'
)


{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 18): <gurobi.Constr *Awaiting 

In [7]:
# Constraint 3: All students for a course should write the exam 

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for r in range(data.n_rooms)
#                           for b,e in covering_sessions(c, t)) <= data.get_course_strength(c)
#               for d in range(data.n_days)
#               for c in range(data.n_courses)
#               for t in range(data.n_times)),
#               name = 'course_enrollment_constraint'
#              )





tt.addConstrs(
    (gp.quicksum(P[c, r, b, e, d] 
                 for r in range(data.n_rooms) 
                 for b, e in covering_sessions(c, t)) <= data.get_course_strength(c)
     for d in range(data.n_days) 
     for c in range(data.n_courses) 
     for t in range(data.n_times)),
    name='course_enrollment_constraint'
)


{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 18): <gurobi.Constr *Awaiting 

In [8]:
# Constraint 4: Student clash constraint

# for each student, for each day, for each time slot, at most one exam
# sigma Ecbed <= 1 for courses enrolled by student s


tt.addConstrs((gp.quicksum(E[c,b,e,d] for c in data.get_courses_student(s) 
                for b,e in covering_sessions(c, t)) <= 1
                for s in range(data.n_students)
                for d in range(data.n_days)
                for t in range(data.n_times)),
              name = 'student_clash_constraint'
             )



{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 18): <gurobi.Constr *Awaiting 

In [9]:
# Constraint 5: No course can have more than 3 rooms alloted
tt.addConstrs(
    (
        # gp.quicksum(signum(C[c, r]) for r in range(data.n_rooms)) <= 3
        gp.quicksum(min(1, C[c, r]) for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

TypeError: '<' not supported between instances of 'Var' and 'int'

## Soft Constraints

In [ ]:
# preferred time slots for exams

## Objective Function

In [ ]:
#objective function: minimize sum of E[c,b,e,d] for all c,b,e,d

objective = E.sum()

tt.setObjective(objective, sense=GRB.MINIMIZE)


In [ ]:
# Maximize Gaps between exams for a student / batch
tt.optimize()